In [22]:
import os
import pandas as pd
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [23]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-19 00:05:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-07-19 00:05:11 (11.4 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [25]:
# Preparing to read the allmay_cleaned.csv, which has been stored on Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
path = "/content/drive/MyDrive/Resources/data/allmay_cleaned.csv"
allmay_cleaned_df = pd.read_csv(path)
# Dataset is now stored in a Pandas Dataframe

In [27]:
allmay_cleaned_df

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,538,2016-05-01 00:00:03,2016-05-01 00:09:02,536,40.741444,-73.975361,497,40.737050,-73.990093,23097,1,1986.0,2
1,224,2016-05-01 00:00:04,2016-05-01 00:03:49,361,40.716059,-73.991908,340,40.712690,-73.987763,23631,1,1977.0,1
2,328,2016-05-01 00:00:14,2016-05-01 00:05:43,301,40.722174,-73.983688,311,40.717227,-73.988021,23049,1,1980.0,1
3,753,2016-05-01 00:00:26,2016-05-01 00:13:00,492,40.750200,-73.990931,228,40.754601,-73.971879,16437,1,1981.0,1
4,511,2016-05-01 00:00:33,2016-05-01 00:09:05,445,40.727408,-73.981420,537,40.740259,-73.984092,20592,1,1991.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6135672,304,2019-05-31 23:59:45.576,2019-06-01 00:04:50.462,3175,40.777480,-73.982886,423,40.765849,-73.986905,32285,1,1987.0,2
6135673,324,2019-05-31 23:59:46.465,2019-06-01 00:05:11.163,2003,40.733812,-73.980544,536,40.741444,-73.975361,33874,1,1994.0,2
6135674,1926,2019-05-31 23:59:51.506,2019-06-01 00:31:58.392,146,40.716250,-74.009106,422,40.770513,-73.988038,33395,1,1985.0,1
6135675,2047,2019-05-31 23:59:52.417,2019-06-01 00:34:00.197,3658,40.743534,-74.003676,2008,40.705693,-74.016777,27624,1,1987.0,0


In [28]:
# Exploring date - cosidering boxing data by lat./long.
allmay_cleaned_df['Start box lat'] = allmay_cleaned_df['start station latitude'].round(3)
allmay_cleaned_df

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Start box lat
0,538,2016-05-01 00:00:03,2016-05-01 00:09:02,536,40.741444,-73.975361,497,40.737050,-73.990093,23097,1,1986.0,2,40.741
1,224,2016-05-01 00:00:04,2016-05-01 00:03:49,361,40.716059,-73.991908,340,40.712690,-73.987763,23631,1,1977.0,1,40.716
2,328,2016-05-01 00:00:14,2016-05-01 00:05:43,301,40.722174,-73.983688,311,40.717227,-73.988021,23049,1,1980.0,1,40.722
3,753,2016-05-01 00:00:26,2016-05-01 00:13:00,492,40.750200,-73.990931,228,40.754601,-73.971879,16437,1,1981.0,1,40.750
4,511,2016-05-01 00:00:33,2016-05-01 00:09:05,445,40.727408,-73.981420,537,40.740259,-73.984092,20592,1,1991.0,1,40.727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6135672,304,2019-05-31 23:59:45.576,2019-06-01 00:04:50.462,3175,40.777480,-73.982886,423,40.765849,-73.986905,32285,1,1987.0,2,40.777
6135673,324,2019-05-31 23:59:46.465,2019-06-01 00:05:11.163,2003,40.733812,-73.980544,536,40.741444,-73.975361,33874,1,1994.0,2,40.734
6135674,1926,2019-05-31 23:59:51.506,2019-06-01 00:31:58.392,146,40.716250,-74.009106,422,40.770513,-73.988038,33395,1,1985.0,1,40.716
6135675,2047,2019-05-31 23:59:52.417,2019-06-01 00:34:00.197,3658,40.743534,-74.003676,2008,40.705693,-74.016777,27624,1,1987.0,0,40.744


In [29]:
# Examine to see how many rows we have for station 422 as a start station
# We'll do our analysis on station 422 in another NB
allmay_cleaned_df[allmay_cleaned_df['start station id'] == 422]

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Start box lat
1152,1101,2016-05-01 07:23:39,2016-05-01 07:42:00,422,40.770513,-73.988038,334,40.742388,-73.997262,18764,1,1985.0,2,40.771
4558,177,2016-05-01 14:59:23,2016-05-01 15:02:20,422,40.770513,-73.988038,3159,40.774925,-73.982666,16841,1,1988.0,2,40.771
5209,550,2016-05-01 15:38:33,2016-05-01 15:47:44,422,40.770513,-73.988038,448,40.756604,-73.997901,24161,1,1996.0,1,40.771
5218,334,2016-05-01 15:38:51,2016-05-01 15:44:25,422,40.770513,-73.988038,3175,40.777480,-73.982886,20773,1,1983.0,1,40.771
5621,537,2016-05-01 16:04:26,2016-05-01 16:13:24,422,40.770513,-73.988038,529,40.757570,-73.990985,19309,1,1980.0,1,40.771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6130980,345,2019-05-31 21:51:37.882,2019-05-31 21:57:22.999,422,40.770513,-73.988038,3699,40.763605,-73.989180,28521,1,1979.0,1,40.771
6131190,1723,2019-05-31 21:57:15.160,2019-05-31 22:25:58.269,422,40.770513,-73.988038,3244,40.731437,-73.994903,28850,1,1996.0,1,40.771
6133420,1393,2019-05-31 22:52:23.707,2019-05-31 23:15:37.242,422,40.770513,-73.988038,412,40.715816,-73.994224,33845,1,1984.0,1,40.771
6135069,241,2019-05-31 23:40:29.694,2019-05-31 23:44:31.144,422,40.770513,-73.988038,3175,40.777480,-73.982886,26925,1,1975.0,1,40.771


Additional preprocessing of allMay dataset

In [30]:
#  Convert 'starttime' column to type datetime from object
allmay_cleaned_df['starttime'] = pd.to_datetime(allmay_cleaned_df['starttime'])


In [9]:
# Split 'starttime' column into 'year', 'month', 'day', hour' and 'minute' columns
allmay_cleaned_df['year'] = allmay_cleaned_df['starttime'].dt.year
allmay_cleaned_df['month'] = allmay_cleaned_df['starttime'].dt.month
allmay_cleaned_df['day'] = allmay_cleaned_df['starttime'].dt.day
allmay_cleaned_df['hour'] = allmay_cleaned_df['starttime'].dt.hour
allmay_cleaned_df['minute'] = allmay_cleaned_df['starttime'].dt.minute


In [10]:
# Drop the 'starttime' and 'stoptime' columns
allmay_cleaned_df.drop(columns=['starttime'], inplace=True)
allmay_cleaned_df.drop(columns=["stoptime"], inplace=True)


In [11]:
# Check values in 'birth year' column
allmay_cleaned_df['birth year'].unique()

array([1986., 1977., 1980., 1981., 1991., 1989., 1998., 1995., 1962.,
       1971., 1987., 1992., 1984., 1985., 1983., 1969., 1997., 1972.,
       1994., 1966., 1967., 1965., 1990., 1978., 1975., 1982., 1970.,
       1954., 1973., 1988., 1963., 1979., 1996., 1952., 1957., 1958.,
       1951., 1950., 1964., 1976., 1961., 1993., 1960., 1974., 1999.,
       1959., 1955., 1968., 1956., 1953., 1900., 1948., 1949., 1938.,
       1945., 1936., 1946., 1944., 1941., 1947., 1932., 1940., 1937.,
       1943., 1942., 2000., 1918., 1885., 1939., 1920., 1910., 1901.,
       1935., 1894., 1899., 1921., 1909., 1926., 1917., 1912., 1934.,
       1925., 1933., 1888., 1930., 1913., 1923., 2001., 1893., 1931.,
       1927., 1886., 1895., 1915., 1896., 1907., 1928., 2002., 1890.,
       1887., 1919., 1929., 1916., 1924., 1889., 1897., 2003., 1905.,
       1911., 1906., 1922.])

In [12]:
# convert 'birth year' column to int from float
allmay_cleaned_df['birth year'] = allmay_cleaned_df['birth year'].astype(int)


In [13]:
allmay_cleaned_df.head()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,usertype,birth year,gender,year,month,day,hour,minute
0,538,536,40.741444,-73.975361,497,40.737050,-73.990093,23097,1,1986,2,2016,5,1,0,0
1,224,361,40.716059,-73.991908,340,40.712690,-73.987763,23631,1,1977,1,2016,5,1,0,0
2,328,301,40.722174,-73.983688,311,40.717227,-73.988021,23049,1,1980,1,2016,5,1,0,0
3,753,492,40.750200,-73.990931,228,40.754601,-73.971879,16437,1,1981,1,2016,5,1,0,0
4,511,445,40.727408,-73.981420,537,40.740259,-73.984092,20592,1,1991,1,2016,5,1,0,0


In [ ]:
###########################
# Machine learning
###########################

In [14]:
# Import ml dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [15]:
# Remove target 'usertype' from features data
y = allmay_cleaned_df['usertype'].ravel()
X = allmay_cleaned_df.drop(columns=["usertype"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)  ##adjust train_size? 

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=40, random_state=42)

# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.958


In [ ]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[  87368   54475]
 [  10360 1381717]]


In [ ]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.89      0.62      0.73    141843
           1       0.96      0.99      0.98   1392077

    accuracy                           0.96   1533920
   macro avg       0.93      0.80      0.85   1533920
weighted avg       0.96      0.96      0.95   1533920



In [ ]:
############
#  Logistic Regression
############

In [16]:
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

In [17]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


 Logistic regression model accuracy: 0.909


In [ ]:
# Prepare confusion matrix for logistic regression
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[   2621  139222]
 [   1006 1391071]]


In [ ]:
# Prepare classificaiton report for logistic regression
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.72      0.02      0.04    141843
           1       0.91      1.00      0.95   1392077

    accuracy                           0.91   1533920
   macro avg       0.82      0.51      0.49   1533920
weighted avg       0.89      0.91      0.87   1533920



In [ ]:
##############
# Neural Network

In [20]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
143805/143805 [==============================] - 220s 2ms/step - loss: 0.1530 - accuracy: 0.9497
Epoch 2/50
143805/143805 [==============================] - 215s 1ms/step - loss: 0.1465 - accuracy: 0.9509
Epoch 3/50
143805/143805 [==============================] - 216s 1ms/step - loss: 0.1459 - accuracy: 0.9510
Epoch 4/50
143805/143805 [==============================] - 216s 2ms/step - loss: 0.1456 - accuracy: 0.9510
Epoch 5/50
143805/143805 [==============================] - 220s 2ms/step - loss: 0.1454 - accuracy: 0.9510
Epoch 6/50
143805/143805 [==============================] - 217s 2ms/step - loss: 0.1454 - accuracy: 0.9511
Epoch 7/50
143805/143805 [==============================] - 213s 1ms/step - loss: 0.1453 - accuracy: 0.9511
Epoch 8/50
143805/143805 [==============================] - 214s 1ms/step - loss: 0.1453 - accuracy: 0.9511
Epoch 9/50
143805/143805 [==============================] - 211s 1ms/step - loss: 0.1452 - accuracy: 0.9511
Epoch 10/50
143805/143805 [=